In [208]:
import os, sys
sys.path.append('lib')
import numpy as np
#from kmeans import kMeans_init_centroids, run_kMeans, find_closest_centroids
from InputDataProcessor import InputDataProcessor 

In [209]:
idp_X=InputDataProcessor(bucket_name='aiml_harveen', use_case='c3w1/anomaly', file_name='X_part2.npy')

In [210]:
idp_Xval=InputDataProcessor(bucket_name='aiml_harveen', use_case='c3w1/anomaly', file_name='X_val_part2.npy')

In [211]:
idp_yval=InputDataProcessor(bucket_name='aiml_harveen', use_case='c3w1/anomaly', file_name='y_val_part2.npy')

In [212]:
X_train=idp_X.loadNumpy()

c3w1/anomaly/orig_data/X_part2.npy


In [213]:
X_val=idp_Xval.loadNumpy()

c3w1/anomaly/orig_data/X_val_part2.npy


In [214]:
y_val=idp_yval.loadNumpy()

c3w1/anomaly/orig_data/y_val_part2.npy


In [215]:
def estimate_gaussian(X): 
    m, n = X.shape
    mu=X.sum(axis=0)/m
    sum=np.zeros(n)
    for elem in range(m):
        sum=sum + (X[elem] - mu)**2
    var=sum/m
    return mu, var

In [216]:
def multivariate_gaussian(X, mu, var):
    """
    Computes the probability 
    density function of the examples X under the multivariate gaussian 
    distribution with parameters mu and var. If var is a matrix, it is
    treated as the covariance matrix. If var is a vector, it is treated
    as the var values of the variances in each dimension (a diagonal
    covariance matrix
    """
    
    k = len(mu)
    
    if var.ndim == 1:
        var = np.diag(var)
        
    X = X - mu
    p = (2* np.pi)**(-k/2) * np.linalg.det(var)**(-0.5) * \
        np.exp(-0.5 * np.sum(np.matmul(X, np.linalg.pinv(var)) * X, axis=1))
    
    return p

In [217]:
def select_threshold(p_val,y_val):
    bestF1=0.0
    bestEpsilon=0.0
    step_size=(max(p_val) - min(p_val))/1000
    for epsilon in np.arange(min(p_val),max(p_val),step_size):
        predictions=np.where(p_val <= epsilon,1,0)
        tp=np.sum(predictions[y_val==1])
        fp=np.sum(predictions[y_val==0])
        fn=np.sum(y_val[predictions==0])
        precision=tp/(tp + fp)
        recall=tp/(tp+fn)
        F1=(2*precision*recall)/(precision + recall)
        if(F1>bestF1):
            bestF1=F1
            bestEpsilon=epsilon
    return bestEpsilon,bestF1

In [218]:
mu,var=estimate_gaussian(X_train)

In [223]:
p_train = multivariate_gaussian(X_train, mu, var)

In [219]:
p_val = multivariate_gaussian(X_val, mu, var)

In [220]:
epsilon,f1=select_threshold(p_val,y_val)

In [221]:
epsilon,f1

(1.3772288907613625e-18, 0.6153846153846154)

In [224]:
print('# Anomalies found: %d'% sum(p_train < epsilon))

# Anomalies found: 117
